# Example of `InferenceData` schema in PyStan
The description of the `InferenceData` structure can be found [here](schema.html).

In [1]:
import arviz as az
import pystan
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
github_commits = data.commits.values
names = data.index.values
N = len(names)
data

,comments,commits,time
Alice,7500,25,4.5
Bob,10100,32,6.0
Cole,18600,49,7.0
Danielle,25200,66,12.0
Erika,27500,96,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = np.array([3.6, 5.1])
N_pred = len(candidate_devs)

In [4]:
linreg_prior_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
}

generated quantities {
    real b0;
    real b1;
    real log_b_sigma;
    real<lower=0> b_sigma;
    
    real c0;
    real c1;
    real log_c_sigma;
    real<lower=0> c_sigma;
    
    vector[N] slack_comments_hat;
    vector[N] github_commits_hat;
    
    b0 = normal_rng(0,200);
    b1 = normal_rng(0,200);
    b_sigma = abs(normal_rng(0,300));
    log_b_sigma = log(b_sigma);
    
    c0 = normal_rng(0,10);
    c1 = normal_rng(0,10);
    c_sigma = fabs(normal_rng(0,6));
    log_c_sigma = log(b_sigma);
    
    for (n in 1:N) {
        slack_comments_hat[n] = normal_rng(b0 + b1 * time_since_joined[n], b_sigma);
        github_commits_hat[n] = normal_rng(c0 + c1 * time_since_joined[n], c_sigma);
    }
}
"""
sm_prior = pystan.StanModel(model_code=linreg_prior_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0d5510d8a96e25d9a54b4530edc08cfc NOW.


In [5]:
linreg_prior_data_dict = {"N": N, "time_since_joined": time_since_joined}
prior = sm_prior.sampling(data=linreg_prior_data_dict, iter=150, chains=1, algorithm='Fixed_param', warmup=0)

In [6]:
linreg_code = """
data {
  int<lower=0> N;
  vector<lower=0>[N] time_since_joined;
  vector<lower=0>[N] slack_comments;
  vector<lower=0>[N] github_commits;
  
  
  // out of sample prediction
  int<lower=0> N_pred;
  vector<lower=0>[N_pred] time_since_joined_pred;
}

parameters {
  real b0;
  real b1;
  real log_b_sigma;
  
  real c0;
  real c1;
  real log_c_sigma;
}

transformed parameters {
  real<lower=0> b_sigma = exp(log_b_sigma);
  real<lower=0> c_sigma = exp(log_c_sigma);
}

model {
  b0 ~ normal(0,200);
  b1 ~ normal(0,200);
  b_sigma ~ normal(0,300);
  slack_comments ~ normal(b0 + b1 * time_since_joined, b_sigma);
  github_commits ~ normal(c0 + c1 * time_since_joined, c_sigma);
  
}

generated quantities {
    // elementwise log likelihood
    vector[N] log_likelihood_slack_comments;
    vector[N] log_likelihood_github_commits;
    
    // posterior predictive
    vector[N] slack_comments_hat;
    vector[N] github_commits_hat;
    
    // out of sample prediction
    vector[N_pred] slack_comments_pred;
    vector[N_pred] github_commits_pred;
    
    // posterior predictive
    for (n in 1:N) {
        log_likelihood_slack_comments[n] = normal_lpdf(slack_comments[n] | b0 + b1 * time_since_joined[n], b_sigma);
        slack_comments_hat[n] = normal_rng(b0 + b1 * time_since_joined[n], b_sigma);
        
        log_likelihood_github_commits[n] = normal_lpdf(github_commits[n] | c0 + c1 * time_since_joined[n], c_sigma);
        github_commits_hat[n] = normal_rng(c0 + c1 * time_since_joined[n], c_sigma);
    }
    
    // out of sample prediction
    for (n in 1:N_pred) {
        slack_comments_pred[n] = normal_rng(b0 + b1 * time_since_joined_pred[n], b_sigma);
        github_commits_pred[n] = normal_rng(c0 + c1 * time_since_joined_pred[n], c_sigma);
    }
}
"""
sm = pystan.StanModel(model_code=linreg_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_bb366f8ad241e32840c67395e3324113 NOW.


In [7]:
linreg_data_dict = {"N": N, "slack_comments": slack_comments, "github_commits": github_commits, "time_since_joined": time_since_joined, "N_pred" : N_pred, "time_since_joined_pred" : candidate_devs_time}
posterior = sm.sampling(data=linreg_data_dict, iter=200, chains=4)

In [8]:
idata_stan = az.from_pystan(
    posterior=posterior,
    prior=prior,
    posterior_predictive=["slack_comments_hat","github_commits_hat"],
    prior_predictive=["slack_comments_hat","github_commits_hat"],
    observed_data=["slack_comments","github_commits"],
    constant_data=["time_since_joined"],
    log_likelihood=["log_likelihood_slack_comments","log_likelihood_github_commits"],
    predictions=["slack_comments_pred", "github_commits_pred"],
    predictions_constant_data=["time_since_joined_pred"],
    coords={"developer": names, "candidate developer" : candidate_devs},
    dims={
        "slack_comments": ["developer"],
        "gitbub_commits" : ["developer"],
        "log_likelihood_slack_comments": ["developer"],
        "log_likelihood_github_commits": ["developer"],
        "slack_comments_hat": ["developer"],
        "github_commits_hat": ["developer"],
        "time_since_joined": ["developer"],
        "slack_comments_pred" : ["candidate developer"],
        "github_commits_pred" : ["candidate developer"],
        "time_since_joined_pred" : ["candidate developer"],
    }
)

In [9]:
idata_stan

Inference data with groups:
	> posterior
	> sample_stats
	> log_likelihood
	> posterior_predictive
	> predictions
	> prior
	> sample_stats_prior
	> prior_predictive
	> observed_data
	> constant_data
	> predictions_constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [10]:
idata_stan.posterior

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 100)
Coordinates:
  * chain        (chain) int32 0 1 2 3
  * draw         (draw) int32 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    b0           (chain, draw) float64 169.1 157.7 215.7 ... 211.9 211.9 167.3
    b1           (chain, draw) float64 1.624e+03 1.619e+03 ... 1.577e+03
    log_b_sigma  (chain, draw) float64 7.429 7.401 7.468 ... 7.288 7.509 7.549
    c0           (chain, draw) float64 0.6469 10.64 1.268 ... -5.759 0.6623
    c1           (chain, draw) float64 5.544 4.693 5.662 ... 6.286 5.21 5.482
    log_c_sigma  (chain, draw) float64 1.951 1.208 2.272 ... 2.235 2.241 1.46
    b_sigma      (chain, draw) float64 1.684e+03 1.638e+03 ... 1.898e+03
    c_sigma      (chain, draw) float64 7.039 3.346 9.695 ... 9.349 9.405 4.306
Attributes:
    created_at:                 2020-01-28T07:43:45.686166
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [11]:
idata_stan.sample_stats

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 100)
Coordinates:
  * chain        (chain) int32 0 1 2 3
  * draw         (draw) int32 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    accept_stat  (chain, draw) float64 0.9967 0.997 0.9654 ... 0.9777 0.9744
    stepsize     (chain, draw) float64 0.06764 0.06764 ... 0.06799 0.06799
    treedepth    (chain, draw) int64 10 10 10 10 6 9 3 10 ... 10 10 10 7 10 4 10
    n_leapfrog   (chain, draw) int64 1023 1023 1023 1023 ... 223 1023 15 1023
    diverging    (chain, draw) bool False False False ... False False False
    energy       (chain, draw) float64 115.4 113.4 115.1 ... 115.8 121.9 118.1
    lp           (chain, draw) float64 -112.0 -112.5 -113.1 ... -116.0 -112.5
Attributes:
    created_at:                 2020-01-28T07:43:45.689166
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [12]:
idata_stan.posterior_predictive

<xarray.Dataset>
Dimensions:             (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain               (chain) int32 0 1 2 3
  * draw                (draw) int32 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 5.158e+03 ... 2.74e+04
    github_commits_hat  (chain, draw, developer) float64 26.36 24.85 ... 94.49
Attributes:
    created_at:                 2020-01-28T07:43:45.695167
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [13]:
idata_stan.predictions

<xarray.Dataset>
Dimensions:              (candidate developer: 2, chain: 4, draw: 100)
Coordinates:
  * chain                (chain) int32 0 1 2 3
  * draw                 (draw) int32 0 1 2 3 4 5 6 7 ... 93 94 95 96 97 98 99
  * candidate developer  (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments_pred  (chain, draw, candidate developer) float64 6.4e+03 ... 6.922e+03
    github_commits_pred  (chain, draw, candidate developer) float64 15.54 ... 26.93
Attributes:
    created_at:                 2020-01-28T07:43:45.697166
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [14]:
idata_stan.observed_data

<xarray.Dataset>
Dimensions:               (developer: 5, github_commits_dim_0: 5)
Coordinates:
  * developer             (developer) object 'Alice' 'Bob' ... 'Erika'
  * github_commits_dim_0  (github_commits_dim_0) int32 0 1 2 3 4
Data variables:
    slack_comments        (developer) int64 7500 10100 18600 25200 27500
    github_commits        (github_commits_dim_0) int64 25 32 49 66 96
Attributes:
    created_at:                 2020-01-28T07:43:45.681167
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [15]:
idata_stan.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2020-01-28T07:43:45.682167
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [16]:
idata_stan.predictions_constant_data

<xarray.Dataset>
Dimensions:                 (candidate developer: 2)
Coordinates:
  * candidate developer     (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    time_since_joined_pred  (candidate developer) float64 3.6 5.1
Attributes:
    created_at:                 2020-01-28T07:43:45.683167
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [17]:
idata_stan.prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 150)
Coordinates:
  * chain        (chain) int32 0
  * draw         (draw) int32 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
Data variables:
    b0           (chain, draw) float64 -359.3 -494.7 379.6 ... 202.5 13.96
    b1           (chain, draw) float64 -102.1 -314.2 -36.77 ... -178.2 222.4
    log_b_sigma  (chain, draw) float64 3.754 5.811 5.603 ... 2.411 3.922 5.885
    b_sigma      (chain, draw) float64 42.68 333.9 271.1 ... 11.14 50.52 359.5
    c0           (chain, draw) float64 5.002 -2.692 10.4 ... -22.78 -8.616 6.764
    c1           (chain, draw) float64 0.3203 1.794 -2.842 ... -16.5 -5.661
    log_c_sigma  (chain, draw) float64 3.754 5.811 5.603 ... 2.411 3.922 5.885
    c_sigma      (chain, draw) float64 3.687 0.05256 11.29 ... 2.15 0.7261 4.593
Attributes:
    created_at:                 2020-01-28T07:43:45.699166
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [18]:
idata_stan.sample_stats_prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 150)
Coordinates:
  * chain        (chain) int32 0
  * draw         (draw) int32 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
Data variables:
    accept_stat  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    created_at:                 2020-01-28T07:43:45.701166
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [19]:
idata_stan.prior_predictive

<xarray.Dataset>
Dimensions:             (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain               (chain) int32 0
  * draw                (draw) int32 0 1 2 3 4 5 6 ... 144 145 146 147 148 149
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 -799.6 ... 4.354e+03
    github_commits_hat  (chain, draw, developer) float64 1.083 1.189 ... -98.43
Attributes:
    created_at:                 2020-01-28T07:43:45.703167
    inference_library:          pystan
    inference_library_version:  2.19.1.1